In [19]:
import torch 
from integrated_gradient import integrated_gradient
from smooth_grad import smooth_gradient
from lrp_module import construct_lrp
from data import get_cifar10_dataset 
import numpy as np 
from tqdm import tqdm 
import seaborn as sns 
import pandas as pd 
sns.set_style("whitegrid")
import os 
import matplotlib.pyplot as plt 
save_path = 'results'
if not os.path.exists(save_path):
    os.makedirs(save_path)


In [66]:
data_train, data_test, trainDataLoader,testDataLoader = get_cifar10_dataset()
model = torch.load('/home/bumjin/deeping_exp/experiments/50_59/exp59/results/model_0.0.pt')
model = model.cuda()

lrp_model = construct_lrp(model, 'cuda:0')


Files already downloaded and verified
Files already downloaded and verified
[+] Finished loading data & Preprocessing
[Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), ReLU(), MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False), Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), ReLU(), MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False), Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), ReLU(), Flatten(start_dim=1, end_dim=-1), Linear(in_features=1024, out_features=512, bias=True), ReLU(), Linear(in_features=512, out_features=10, bias=True)]


In [67]:
from scipy.stats import kendalltau
from scipy.spatial.distance import cosine
def kendall(a,b):
    tau, p_value = kendalltau(a,b, variant='c') # variant : [b,c]
    return tau

def cosine_sim(a,b):
    a = a / np.linalg.norm(a)
    b = b / np.linalg.norm(b)
    return 1 - cosine(a,b) # cosine distance

NUM_SAMPLES = 50

results = {
    'cosine' : [],
    'kendall' : [],
    'index' : [],
    'y' : [], 
    'perturb': [],
    'method' : []
}

def lrp_wrapper(model, x,y):
    return lrp_model.forward(x.unsqueeze(0), y=y)['R'][0]
    
def smooth_wrapper(model, x,y):
    return smooth_gradient(model, x, 150, 0.15, y)
def ig_wrapper(model, x,y):
    return integrated_gradient(model, x, 150, y, baseline=torch.zeros_like(x).cuda())

for method in ['lrp', 'smooth', 'ig']:
    fn = {'smooth':smooth_wrapper, 'ig':ig_wrapper, 'lrp':lrp_wrapper,}[method]
    for i in tqdm(range(NUM_SAMPLES)):
        x,y = data_test[i]
        # ---------------
        x = x.cuda()
        output = fn(model, x, y)
        non_perturbed = output.cpu().detach().numpy().flatten()
        for eps in [i/10 for i in range(1, 11)]:
            x_hat = x + (1-torch.rand_like(x)) * eps
            output = fn(model, x_hat, y)
            perturbed = output.cpu().detach().numpy().flatten()
            results['cosine'].append(cosine_sim(non_perturbed, perturbed))
            results['kendall'].append(kendall(non_perturbed, perturbed))
            results['perturb'].append(eps)
            results['index'].append(i)
            results['y'].append(y)
            results['method'].append(method)
        
        if i>NUM_SAMPLES:
            break 
df = pd.DataFrame(results)


100%|██████████| 50/50 [00:05<00:00,  8.92it/s]


In [70]:
sns.relplot(x='cosine', y='kendall', data=df, hue='method', palette='Set1', col='perturb', col_wrap=5, height=2, alpha=1.0, s=10, edgecolor=None)
plt.savefig(os.path.join(save_path, 'eps_perturb.pdf'))

In [71]:
sns.relplot(x='cosine', y='kendall', data=df, hue='perturb', palette='plasma', col='method', height=2.5, alpha=1.0, s=8, edgecolor = None)
plt.savefig(os.path.join(save_path, 'eps_method.pdf'))


# Masking Pixels

In [72]:
import torch 

def zero_masking(x, filter, ratio):
    assert x[0].size() == filter.size(), f"{x.size()} - {filter.size()}"
    # masking the region by maximum regions in the filter. 
    # filter is a patch x patch region with magnitude
    filter = filter.flatten()
    v, index = torch.sort(filter, descending=True)
    size = x.size()
    x = x.reshape(x.size(0), -1)
    for i in range(int(x.size(1)*ratio)):
        idx = index[i]
        x[0, idx].zero_()
    x = x.reshape(*size)
    return x

In [73]:
from scipy.stats import kendalltau
from scipy.spatial.distance import cosine
def kendall(a,b):
    tau, p_value = kendalltau(a,b, variant='c') # variant : [b,c]
    return tau

def cosine_sim(a,b):
    a = a / np.linalg.norm(a)
    b = b / np.linalg.norm(b)
    return 1 - cosine(a,b) # cosine distance

NUM_SAMPLES = 50

results = {
    'cosine' : [],
    'kendall' : [],
    'index' : [],
    'y' : [], 
    'remove_portion': [],
    'method' : []
}

def lrp_wrapper(model, x,y):
    return lrp_model.forward(x.unsqueeze(0), y=y)['R'][0]
    
def smooth_wrapper(model, x,y):
    return smooth_gradient(model, x, 150, 0.15, y)
def ig_wrapper(model, x,y):
    return integrated_gradient(model, x, 150, y, baseline=torch.zeros_like(x).cuda())

for method in ['lrp', 'smooth', 'ig']:
    fn = {'smooth':smooth_wrapper, 'ig':ig_wrapper, 'lrp':lrp_wrapper,}[method]
    for i in tqdm(range(NUM_SAMPLES)):
        x,y = data_test[i]
        # ---------------
        x = x.cuda()
        output = fn(model, x, y)
        non_perturbed = output.cpu().detach().numpy().flatten()
        for eps in [i/10 for i in range(1, 11)]:
            x_hat = zero_masking(x, torch.abs(output).mean(dim=0), eps)
            output = fn(model, x_hat, y)
            perturbed = output.cpu().detach().numpy().flatten()
            results['cosine'].append(cosine_sim(non_perturbed, perturbed))
            results['kendall'].append(kendall(non_perturbed, perturbed))
            results['remove_portion'].append(eps)
            results['index'].append(i)
            results['y'].append(y)
            results['method'].append(method)
        
        if i>NUM_SAMPLES:
            break 
df = pd.DataFrame(results)


100%|██████████| 50/50 [00:11<00:00,  4.52it/s]


In [76]:
sns.relplot(x='cosine', y='kendall', data=df, hue='method', palette='Set1', col='remove_portion', col_wrap=5, height=2, alpha=1.0, s=10, edgecolor=None)
# plt.show()
plt.savefig(os.path.join(save_path, 'remove_portion.pdf'))

In [77]:
sns.relplot(x='cosine', y='kendall', data=df, hue='remove_portion', palette='plasma', col='method', height=2.5, alpha=1.0, s=8, edgecolor = None )
# plt.show()
plt.savefig(os.path.join(save_path, 'remove_method.pdf'))
